In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import glob

import pandas as pd

In [3]:
## delete the folders not in live config ( extra crap )
live_config_files = glob.glob('cfgs_live/*', recursive=True)
allowed_symbols = []
symbol_files = {}
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    allowed_symbols.append(symbol)
    symbol_files[symbol] = f

# static_configs = {}
# for symbol in symbol_files:
#     file = symbol_files[symbol]
#     with open(file) as f:
#         data = json.load(f)
#     config_name = data['config_name']
#
#     if 'static' in config_name:
#         t = os.path.getmtime(file)
#         modification_time = datetime.datetime.fromtimestamp(t)
#         static_configs[symbol] = modification_time
#
# #print (static_configs)
# old_configs_if_before = datetime.datetime(2022, 3, 31)
# static_configs_new = [x for x in static_configs if static_configs[x] < old_configs_if_before]
#
# #print(static_configs_new)
# x = ''
# for c in static_configs_new:
#     x += c+'\n'
# with open('next_batch_config','w') as f:
#     f.write(x)
#
# #allowed_symbols = ["ALICEUSDT","XRPUSDT","HOTUSDT"]

In [4]:
# ## NOW copy result folders :)
# import json
# import shutil
#
# files = glob.iglob('backtests/binance/' + '**/live_config.json', recursive=True)
#
# for filename in files:
#     with open(filename) as f:
#         config = json.load(f)
#     symbol = config['config_name'].split('_')[2]
#
#     if symbol not in symbol_files: continue
#
#     with open(symbol_files[symbol]) as f:
#         production_config = json.load(f)
#
#     long_pc = production_config['long']
#     long_c = config['long']
#
#     k_pc = long_pc.keys()
#     k_c = long_c.keys()
#
#     if len(k_c) != len(k_c): continue
#
#     same = True
#     for k in k_pc:
#         if k not in k_c:
#             same = False
#             break
#         v_pc = long_pc[k]
#         v_c = long_c[k]
#         #print(f'{k}={v_c},{v_pc}')
#
#     if not same: continue
#
#     print(f'{filename} is same as {symbol_files[symbol]}')
#
#     src = filename.replace('\\live_config.json', '')
#     dst = f'backtests/binance_production/{symbol}'
#     shutil.copytree(src=src, dst=dst, dirs_exist_ok=True)



backtests/binance\1000SHIBUSDT\plots\2022-04-03T194816\live_config.json is same as cfgs_live\1000SHIB.json
backtests/binance\1000SHIBUSDT\plots\2022-04-06T180851\live_config.json is same as cfgs_live\1000SHIB.json
backtests/binance\1000XECUSDT\plots\2022-04-11T104928\live_config.json is same as cfgs_live\1000XEC.json
backtests/binance\1INCHUSDT\plots\2022-04-08T091942\live_config.json is same as cfgs_live\1INCH.json
backtests/binance\ADAUSDT\plots\2022-04-03T202719\live_config.json is same as cfgs_live\ADA.json
backtests/binance\ADAUSDT\plots\2022-04-06T185658\live_config.json is same as cfgs_live\ADA.json
backtests/binance\AKROUSDT\plots\2022-04-06T190136\live_config.json is same as cfgs_live\AKRO.json
backtests/binance\AKROUSDT\plots\2022-04-10T225650\live_config.json is same as cfgs_live\AKRO.json
backtests/binance\ALGOUSDT\plots\2022-04-08T070001\live_config.json is same as cfgs_live\ALGO.json
backtests/binance\ALICEUSDT\plots\2022-04-05T084248\live_config.json is same as cfgs_live

In [4]:
import requests
prices_url = "https://fapi.binance.com/fapi/v1/ticker/price"
pricedata = requests.get(prices_url).json()
volume_url = "https://fapi.binance.com//fapi/v1/ticker/24hr"
volume_data = requests.get(volume_url).json()
data_url = "https://fapi.binance.com/fapi/v1/exchangeInfo"
data = requests.get(data_url).json()

symbol_notional_volume = {}

for datas in data["symbols"]:
    if "USDT" in datas["pair"]:
        ts = datas['onboardDate']
        listed_on = datetime.datetime.utcfromtimestamp(ts / 1000).strftime(
            '%Y-%m-%d %H:%M:%S')
        symbol = datas["pair"]

        if symbol not in allowed_symbols:
            continue

        min_qty = float(datas["filters"][1]["minQty"])
        min_notional_fixed = float(datas["filters"][5]["notional"])

        price = 0
        for i in pricedata:
            if i["symbol"] == symbol:
                price = i["price"]

        for i in volume_data:
            if i["symbol"] == symbol:
                volm = str((int(float(i["quoteVolume"]) / 1000000))).zfill(4)

        min_notional_calc = min_qty * float(price)

        if min_notional_calc <= min_notional_fixed:
            min_notional = min_notional_fixed
        else:
            min_notional = min_notional_calc

        symbol_notional_volume[symbol] = {
            "vol":volm,
            "listed":listed_on,
            "notion":min_notional
        }

symbol_notional_volume


{'ETHUSDT': {'vol': '5326', 'listed': '2022-03-21 08:00:00', 'notion': 5.0},
 'BCHUSDT': {'vol': '0079', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'XRPUSDT': {'vol': '0398', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'EOSUSDT': {'vol': '0121', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'LTCUSDT': {'vol': '0149', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'TRXUSDT': {'vol': '0051', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'ETCUSDT': {'vol': '0433', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'LINKUSDT': {'vol': '0172', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'XLMUSDT': {'vol': '0034', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'ADAUSDT': {'vol': '0512', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'XMRUSDT': {'vol': '0316', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'DASHUSDT': {'vol': '0032', 'listed': '2019-09-25 08:00:00', 'notion': 5.0},
 'ZECUSDT': {'vol': '0095', 'listed': '2019-09-25 08:00:00', 'notion': 5.0

In [5]:
## get current config
current_config = pd.read_csv('run_config.csv')

config_data = {}
for symbol in allowed_symbols:
    c = current_config.loc[current_config['symbol'] == symbol.replace('USDT','')]
    config_data[symbol]={
        'lm':c['long_mode'].values[0],
        'sm':c['short_mode'].values[0],
        'le':c['long_exposure'].values[0],
        'se':c['short_exposure'].values[0],
    }
config_data

{'1000SHIBUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.035, 'se': 0.0},
 '1000XECUSDT': {'lm': 'n', 'sm': 'n', 'le': 0.035, 'se': 0.02},
 '1INCHUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.035, 'se': 0.0},
 'ADAUSDT': {'lm': 'n', 'sm': 'n', 'le': 0.035, 'se': 0.02},
 'AKROUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.035, 'se': 0.0},
 'ALGOUSDT': {'lm': 'n', 'sm': 'n', 'le': 0.035, 'se': 0.02},
 'ALICEUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.035, 'se': 0.0},
 'ALPHAUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.035, 'se': 0.0},
 'ANCUSDT': {'lm': 'n', 'sm': 'n', 'le': 0.03, 'se': 0.02},
 'ANKRUSDT': {'lm': 'n', 'sm': 'n', 'le': 0.035, 'se': 0.02},
 'API3USDT': {'lm': 'n', 'sm': 'n', 'le': 0.035, 'se': 0.03},
 'ARUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.035, 'se': 0.0},
 'ATOMUSDT': {'lm': 'n', 'sm': 'n', 'le': 0.035, 'se': 0.02},
 'BAKEUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.035, 'se': 0.0},
 'BANDUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.035, 'se': 0.0},
 'BCHUSDT': {'lm': 'n', 'sm': 'p', 'le': 0.03, 'se': 0.0},
 'BELUSDT': {'

In [12]:
import warnings

warnings.filterwarnings("ignore")

big_data = pd.DataFrame()
files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
i = 0
#print(symbol_notional_volume)
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance_production', '')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Production Config':
                    d['prod'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        # # symbol_notional_volume[symbol] = {
        # #     "vol":volm,
        # #     "listed":listed_on,
        # #     "notion":min_notional
        # # }
        symbol = d['s']
        d['vol'] = symbol_notional_volume[symbol]['vol']
        d['listed'] = symbol_notional_volume[symbol]['listed']
        d['notion'] = symbol_notional_volume[symbol]['notion']
        # # config_data[symbol]={
        # #     'lm':c['long_mode'].values[0],
        # #     'sm':c['short_mode'].values[0],
        # #     'le':c['long_exposure'].values[0],
        # #     'se':c['short_exposure'].values[0],
        # # }
        d['lm'] = config_data[symbol]['lm']
        d['sm'] = config_data[symbol]['sm']
        d['le'] = config_data[symbol]['le']
        d['se'] = config_data[symbol]['se']
        # print(d)
        big_data = big_data.append(d, ignore_index=True)

big_data

,file,v,prod,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,...,bnkrs_%,ebrs,maxss,vol,listed,notion,lm,sm,le,se
0,\1000SHIBUSDT\backtest_result.txt,5.5,True,1000SHIBUSDT,True,90.63,0.2460,0.065637,0.629854,100.00,...,5.959,0.08869,180.079167,3817,2021-05-10 07:00:00,5.0,n,p,0.035,0.00
1,\1000XECUSDT\backtest_result.txt,5.5,False,1000XECUSDT,True,43.36,0.1830,0.064167,0.648708,100.00,...,100.000,0.75270,39.361125,0027,2021-09-16 07:00:00,5.0,n,n,0.035,0.02
2,\1INCHUSDT\backtest_result.txt,5.5,False,1INCHUSDT,True,71.61,0.1980,0.059845,0.592445,100.00,...,44.480,0.44830,33.708333,0039,2020-12-25 07:00:00,5.0,n,p,0.035,0.00
3,\ADAUSDT\backtest_result.txt,5.5,True,ADAUSDT,True,26.33,0.0809,0.051500,0.391366,100.00,...,75.970,0.64290,39.211792,0512,2019-09-25 08:00:00,5.0,n,n,0.035,0.02
4,\AKROUSDT\backtest_result.txt,5.5,False,AKROUSDT,True,63.61,0.1680,0.095610,0.746314,100.00,...,91.780,0.63320,28.277792,0014,2021-01-18 07:00:00,5.0,n,p,0.035,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,\XTZUSDT\backtest_result.txt,5.5,False,XTZUSDT,True,48.20,0.1510,0.080333,0.555208,100.00,...,99.530,0.72280,38.434042,0078,2019-09-25 08:00:00,5.0,n,p,0.030,0.00
92,\ZECUSDT\backtest_result.txt,5.5,True,ZECUSDT,True,64.52,0.1770,0.074299,0.590802,100.00,...,88.210,0.65640,36.830542,0095,2019-09-25 08:00:00,5.0,n,p,0.035,0.00
93,\ZENUSDT\backtest_result.txt,5.5,False,ZENUSDT,True,61.20,0.1830,0.107443,0.877769,95.61,...,18.190,0.22950,49.320833,0019,2019-09-25 08:00:00,5.0,n,p,0.035,0.00
94,\ZILUSDT\backtest_result.txt,5.5,False,ZILUSDT,True,24.10,0.0730,0.077813,0.534668,100.00,...,5.950,0.07147,33.434708,0577,2019-09-25 08:00:00,5.0,n,p,0.035,0.00


Now filter the dataframe

In [14]:
#big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%'] if
df = big_data[(big_data.s.isin(allowed_symbols)) ]
df

,file,v,prod,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,...,ebrs,maxss,vol,listed,notion,lm,sm,le,se,adgt_%
0,\1000SHIBUSDT\backtest_result.txt,5.5,True,1000SHIBUSDT,True,90.63,0.2460,0.065637,0.629854,100.00,...,0.08869,180.079167,3817,2021-05-10 07:00:00,5.0,n,p,0.035,0.00,-1.39400
1,\1000XECUSDT\backtest_result.txt,5.5,False,1000XECUSDT,True,43.36,0.1830,0.064167,0.648708,100.00,...,0.75270,39.361125,0027,2021-09-16 07:00:00,5.0,n,n,0.035,0.02,0.35900
2,\1INCHUSDT\backtest_result.txt,5.5,False,1INCHUSDT,True,71.61,0.1980,0.059845,0.592445,100.00,...,0.44830,33.708333,0039,2020-12-25 07:00:00,5.0,n,p,0.035,0.00,0.37300
3,\ADAUSDT\backtest_result.txt,5.5,True,ADAUSDT,True,26.33,0.0809,0.051500,0.391366,100.00,...,0.64290,39.211792,0512,2019-09-25 08:00:00,5.0,n,n,0.035,0.02,0.17700
4,\AKROUSDT\backtest_result.txt,5.5,False,AKROUSDT,True,63.61,0.1680,0.095610,0.746314,100.00,...,0.63320,28.277792,0014,2021-01-18 07:00:00,5.0,n,p,0.035,0.00,0.19900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,\XTZUSDT\backtest_result.txt,5.5,False,XTZUSDT,True,48.20,0.1510,0.080333,0.555208,100.00,...,0.72280,38.434042,0078,2019-09-25 08:00:00,5.0,n,p,0.030,0.00,0.18140
92,\ZECUSDT\backtest_result.txt,5.5,True,ZECUSDT,True,64.52,0.1770,0.074299,0.590802,100.00,...,0.65640,36.830542,0095,2019-09-25 08:00:00,5.0,n,p,0.035,0.00,0.18238
93,\ZENUSDT\backtest_result.txt,5.5,False,ZENUSDT,True,61.20,0.1830,0.107443,0.877769,95.61,...,0.22950,49.320833,0019,2019-09-25 08:00:00,5.0,n,p,0.035,0.00,0.51300
94,\ZILUSDT\backtest_result.txt,5.5,False,ZILUSDT,True,24.10,0.0730,0.077813,0.534668,100.00,...,0.07147,33.434708,0577,2019-09-25 08:00:00,5.0,n,p,0.035,0.00,-1.56700


In [16]:
# df.drop_duplicates(
#     subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
#             'short'], keep='first',
#     inplace=True)
dx = df[['s','lm','sm','notion','le','se','vol', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

,s,lm,sm,notion,le,se,vol,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
0,1000SHIBUSDT,n,p,5.0,0.035,0.00,3817,0.2460,-1.64000,100.00,5.959,0.7673,0.08869,12.442375,0.629854,0.065637,180.079167,0.747041,0.124514
1,1000XECUSDT,n,n,5.0,0.035,0.02,0027,0.1830,0.17600,100.00,100.000,0.7694,0.75270,10.770125,0.648708,0.064167,39.361125,0.541715,0.086314
2,1INCHUSDT,n,p,5.0,0.035,0.00,0039,0.1980,0.17500,100.00,44.480,0.7476,0.44830,7.603458,0.592445,0.059845,33.708333,0.620962,0.125995
3,ADAUSDT,n,n,5.0,0.035,0.02,0512,0.0809,0.09610,100.00,75.970,0.8072,0.64290,6.515292,0.391366,0.051500,39.211792,0.519449,0.101922
4,AKROUSDT,n,p,5.0,0.035,0.00,0014,0.1680,0.03100,100.00,91.780,0.9070,0.63320,9.093750,0.746314,0.095610,28.277792,0.536115,0.115026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,XTZUSDT,n,p,5.0,0.030,0.00,0078,0.1510,0.03040,100.00,99.530,0.8182,0.72280,16.641667,0.555208,0.080333,38.434042,0.427126,0.107084
92,ZECUSDT,n,p,5.0,0.035,0.00,0095,0.1770,0.00538,100.00,88.210,0.8301,0.65640,15.332625,0.590802,0.074299,36.830542,0.492966,0.138920
93,ZENUSDT,n,p,5.0,0.035,0.00,0019,0.1830,0.33000,95.61,18.190,0.7091,0.22950,11.806250,0.877769,0.107443,49.320833,0.613587,0.140844
94,ZILUSDT,n,p,5.0,0.035,0.00,0577,0.0730,-1.64000,100.00,5.950,0.7921,0.07147,13.360417,0.534668,0.077813,33.434708,0.780698,0.125380


In [17]:
ds = dx[(dx['sm'] == 'n') | (dx['sm'] == 'gs')]
ds

,s,lm,sm,notion,le,se,vol,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
1,1000XECUSDT,n,n,5.0,0.035,0.02,0027,0.1830,0.1760,100.0,100.00,0.7694,0.7527,10.770125,0.648708,0.064167,39.361125,0.541715,0.086314
3,ADAUSDT,n,n,5.0,0.035,0.02,0512,0.0809,0.0961,100.0,75.97,0.8072,0.6429,6.515292,0.391366,0.051500,39.211792,0.519449,0.101922
5,ALGOUSDT,n,n,5.0,0.035,0.02,0078,0.1860,0.1400,100.0,88.80,0.8084,0.6390,17.899292,0.936239,0.103523,27.758333,0.503807,0.092130
8,ANCUSDT,n,n,5.0,0.030,0.02,0045,0.1140,0.6780,100.0,100.00,0.9220,0.9457,12.086792,0.294680,0.084670,10.819458,0.159975,0.037467
9,ANKRUSDT,n,n,5.0,0.035,0.02,0175,0.1690,0.0184,100.0,95.05,0.8392,0.6541,12.125000,0.547022,0.088469,14.161125,0.520499,0.063347
10,API3USDT,n,n,5.0,0.035,0.03,0021,0.6330,0.3480,100.0,100.00,0.9257,0.8856,3.054863,0.240547,0.049982,13.779167,0.287792,0.066013
12,ATOMUSDT,n,n,5.0,0.035,0.02,0244,0.1760,0.1270,100.0,64.62,0.9162,0.5639,16.588208,0.489647,0.081672,35.834708,0.577798,0.119612
16,BELUSDT,n,n,5.0,0.035,0.02,0015,0.3140,0.1060,100.0,92.08,0.8391,0.6465,9.887500,0.562763,0.098852,13.455542,0.504318,0.080861
17,BLZUSDT,n,n,5.0,0.035,0.02,0021,0.2540,0.0843,100.0,100.00,0.8477,0.7014,7.234708,0.740899,0.082807,27.396542,0.489913,0.120388
19,BTSUSDT,n,n,5.0,0.035,0.02,0009,0.1740,0.0881,100.0,100.00,0.8957,0.7647,20.911792,0.981370,0.138583,30.752083,0.439665,0.093844


In [32]:
ds_proposed = dx[
    #(dx['sm'] == 'n') &
    (dx['bnkrs_%'] == 100) &
    # (dx['ebrs'] > 0.7) &
    (dx['pastds'] < 0.1)
    #(dx['maxss'] < 35)
    ]
ds_proposed = ds_proposed[['s','sm','notion','le','se','vol', 'adgs_%', 'bnkrs_%',  'ebrs','maxss', 'pamaxs','pastds' ]]
ds_proposed

,s,sm,notion,le,se,vol,adgs_%,bnkrs_%,ebrs,maxss,pamaxs,pastds
1,1000XECUSDT,n,5.0,0.035,0.02,0027,0.1760,100.0,0.7527,39.361125,0.541715,0.086314
8,ANCUSDT,n,5.0,0.030,0.02,0045,0.6780,100.0,0.9457,10.819458,0.159975,0.037467
10,API3USDT,n,5.0,0.035,0.03,0021,0.3480,100.0,0.8856,13.779167,0.287792,0.066013
19,BTSUSDT,n,5.0,0.035,0.02,0009,0.0881,100.0,0.7647,30.752083,0.439665,0.093844
20,C98USDT,n,5.0,0.035,0.02,0146,0.2620,100.0,0.8572,28.186125,0.402502,0.095534
24,COMPUSDT,n,5.0,0.035,0.02,0096,0.1910,100.0,0.8382,28.412500,0.398611,0.079196
31,DOGEUSDT,n,5.0,0.035,0.02,0679,0.1270,100.0,0.6884,28.373625,0.503842,0.085994
36,ENSUSDT,n,5.0,0.030,0.02,0025,0.3990,100.0,0.8914,5.195833,0.227052,0.049809
37,EOSUSDT,p,5.0,0.030,0.00,0121,0.0561,100.0,0.7360,39.181958,0.441473,0.085832
41,FLOWUSDT,n,5.0,0.035,0.02,0033,0.0905,100.0,0.8292,39.319458,0.372590,0.090211


ds = dx[
   (dx['adgs_%'] > 0.00)
  #& (dx['bnkrs_%'] == 100.00)
]
ds = ds[['file', 'prod', 's', 'adgs_%', 'bnkrs_%','ebrs','maxss', 'pamaxs','pastds' ]]
ds

In [69]:
short_symbols = ds['s'].tolist()
print(short_symbols)
config = pd.read_csv('run_config.csv')
config['usdt'] = config['symbol']+'USDT'
config = config[(config.usdt.isin(short_symbols))]
config

['1000XECUSDT', '1INCHUSDT', 'ADAUSDT', 'AKROUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ANKRUSDT', 'API3USDT', 'ARUSDT', 'ATOMUSDT', 'BAKEUSDT', 'BELUSDT', 'BLZUSDT', 'BNBUSDT', 'BTSUSDT', 'C98USDT', 'CHZUSDT', 'COMPUSDT', 'DENTUSDT', 'DGBUSDT', 'DODOUSDT', 'DOGEUSDT', 'DOTUSDT', 'DUSKUSDT', 'DYDXUSDT', 'ENJUSDT', 'ETCUSDT', 'ETHUSDT', 'FLMUSDT', 'FLOWUSDT', 'GRTUSDT', 'HBARUSDT', 'HOTUSDT', 'ICPUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'KAVAUSDT', 'KLAYUSDT', 'KNCUSDT', 'LINKUSDT', 'LITUSDT', 'LTCUSDT', 'MATICUSDT', 'MKRUSDT', 'MTLUSDT', 'NEOUSDT', 'OGNUSDT', 'OMGUSDT', 'ONTUSDT', 'QTUMUSDT', 'RLCUSDT', 'ROSEUSDT', 'RVNUSDT', 'SCUSDT', 'SNXUSDT', 'STMXUSDT', 'SUSHIUSDT', 'SXPUSDT', 'TLMUSDT', 'TOMOUSDT', 'TRBUSDT', 'UNFIUSDT', 'VETUSDT', 'XLMUSDT', 'XMRUSDT', 'XRPUSDT', 'ZECUSDT', 'ZENUSDT', 'ZRXUSDT']


,Unnamed: 0,symbol,direction,long_mode,short_mode,long_exposure,short_exposure,usdt
0,0,ALICE,long,n,p,0.035,0.00,ALICEUSDT
1,1,IOST,long,n,p,0.035,0.00,IOSTUSDT
2,2,OMG,long,n,p,0.035,0.00,OMGUSDT
3,3,ATOM,long_short,n,n,0.035,0.02,ATOMUSDT
4,4,KAVA,long,n,p,0.035,0.00,KAVAUSDT
...,...,...,...,...,...,...,...,...
80,80,MATIC,long_short,n,n,0.035,0.02,MATICUSDT
81,81,LTC,long_short,n,n,0.035,0.02,LTCUSDT
82,82,LINK,long_short,n,n,0.035,0.02,LINKUSDT
83,83,GRT,long_short,n,n,0.035,0.02,GRTUSDT


## find the ones that are not supposed to be shorted!
short_symbols = ds['s'].tolist()

config = pd.read_csv('run_config.csv')
config['usdt'] = config['symbol']+'USDT'
config = config[(~config.usdt.isin(short_symbols)) & (config['short_mode'] == 'n')]
## find the risky ones!
not_supposed_to_be_shorted = config['usdt'].tolist()
not_supposed_to_be_shorted

dx
# ss = dx['s'].tolist()
# ss
ds_not_good = dx[(dx.s.isin(not_supposed_to_be_shorted))]
#s_not_good
#ds_not_good
ds_not_good